# Programming Assignment #4: Checksums

In this assignment, we will explore various uses of modular arithmetic to construct checksums, which are used extensively in error-detection and error-correction codes.

As a general rule, there are two types of errors that can be detected or even better corrected. The first type is a change in one of the digits, e.g., a 7 can be changed to a 5. The sercond type is a swap of two adjacent digits, e.g., 147 can change to 417. Different techniques will detect and/or fix some of these errors.


## Airline Ticket Numbers

When you buy a plane ticket, you may find a ticket id (different from your confirmation number). This is a 15-digit number, where the 15th digit is a checksum. In particular, the 15th digit is chosen such that it is equal to the remainder of the 14-digit number (of the first 14 digits) modulo 7. For example, the following is a valid airline ticket number:

$$1 2 3 4 5 6 7 8 9 0 1 2 3 4 0$$

because $12345678901234 \bmod 7 = 0$.

### TODO-1 (10 points)

Write the function `(valid-ticket ID)` that verifies that the given ID, which is a list of 14 digits, is a valid airline ticket ID. For example,

    (valid-ticket '(1 2 3 4 5 6 7 8 9 0 1 2 3 4 0)) = T
    
> Hint: I found it useful to write a function that extracts the ticket number from a ticket. E.g., the ticket number of the ticket above is 12,345,678,901,234.

> Hint: You will need to define a data type for things like "digit" and "list of digits". Later, we will bd writing functions for things like "lists of natural" and try to pass a list of digits as an argument. Unfortunately, ACL2 will not allow this. Let's simplify things right now by telling ACL2 using `defthm` that a list of digits **is** a list of naturals. This will save you a lot of time later, so just do it now and forget it.

In [1]:
(defsnapshot from-the-top)
(defsnapshot todo-1)

(defdata digit (range integer (0 <= _ <= 9)))
(defdata digit-list (listof digit))
(defdata airline-ticket (list digit digit digit digit digit
                              digit digit digit digit digit
                              digit digit digit digit digit))

(defthm nat-listp-digit-listp
    (implies (digit-listp lst)
             (nat-listp lst)))

(definec ticket-number (ticket :digit-list) :nat
    (if (endp ticket)
        0
        (if (endp (rest ticket))
            0
            (+ (* (first ticket) (expt 10 (- (len ticket) 2)))
               (ticket-number (rest ticket))))))
               
(check-expect (ticket-number '(1 2 3 4 5)) 1234)

(definec valid-ticket (ticket :digit-list) :bool
    (and (equal (len ticket) 15)
         (equal (mod (ticket-number ticket) 7) (first (last ticket))))
    )

(check-expect (valid-ticket '(1 2 3 4 5 6 7 8 9 0 1 2 3 4 0)) t)

(check-expect (valid-ticket '(1 2)) nil)





ACL2S !>>(DEFSNAPSHOT FROM-THE-TOP)
          20:x(DEFMACRO DEFSNAPSHOT (LABEL) ...)

Summary
Form:  ( DEFLABEL FROM-THE-TOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FROM-THE-TOP
ACL2S !>>(DEFSNAPSHOT TODO-1)

Summary
Form:  ( DEFLABEL TODO-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-1
ACL2S !>>(DEFDATA DIGIT (RANGE INTEGER (0 <= _ <= 9)))
 Predicate events...
Form:  ( DEFUN DIGITP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (DIGITP ACL2::V1) <= body -- not complete. 
Reasons: ("Illegal tau rule") 
 (DIGITP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules.")

Form:  ( DEFTHM DIGIT=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-DIGIT-BUILTIN ...)
Form:  ( DEFUN NTH-DIGIT/ACC-BUILTIN ...)
Form:  ( PROGN (

### TODO-2

We are interested in detecting errors, e.g., an invalid ticket number. What we want to show is that if a ticket number was written down incorrectly, e.g., with one of the common mistakes (changing a number or transposing two adjacent numbers), then the resulting ticket will be invalid.

To get started, we need to write functions that perform those two types of errors. I.e., write a function `(change-nth-digit-to ticket n d)` that returns a new ticket that is equal to the old ticket, except that the nth digit will be `d`. Also, write a function called `(transpose-nth-digit ticket n)` that returns a ticket where the nth digit is swapped with the *next* digit. 

> **Important:** We will be using these functions later, so don't write them to work just for airline tickets. Write them so they work for arbitrary lists of digits instead. 

> Note: If there are $N$ digits in a ticket, `n` should be between 0 and $n-1$ for `change-nth-digit-to` and between 0 and $n-2$ for `transpose-nth-digit`.

In [2]:
(defsnapshot todo-2)

(definec change-nth-digit-to (msg :digit-list n :nat d :digit) :digit-list
    (if (endp msg)
        msg
        (if (zp n)
            (cons d (rest msg))
            (cons (first msg) (change-nth-digit-to (rest msg) (1- n) d)))))
            
(check-expect (change-nth-digit-to '(1 2 3 4 5) 3 0) '(1 2 3 0 5))

(definec transpose-nth-digit (msg :digit-list n :nat) :digit-list
    (if (or (endp msg) (endp (rest msg)))
        msg
        (if (zp n)
            (cons (second msg) 
                  (cons (first msg) 
                        (rest (rest msg))))
            (cons (first msg) (transpose-nth-digit (rest msg) (1- n))))))
            
(check-expect (transpose-nth-digit '(1 2 3 4 5) 2) '(1 2 4 3 5))



ACL2S !>>(DEFSNAPSHOT TODO-2)

Summary
Form:  ( DEFLABEL TODO-2 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-2
ACL2S !>>(DEFINEC CHANGE-NTH-DIGIT-TO
                  (MSG DIGIT-LIST N NAT D DIGIT)
                  DIGIT-LIST
                  (IF (ENDP MSG)
                      MSG
                      (IF (ZP N)
                          (CONS D (REST MSG))
                          (CONS (FIRST MSG)
                                (CHANGE-NTH-DIGIT-TO (REST MSG)
                                                     (1- N)
                                                     D)))))

Form:  ( TEST-DEFINITION CHANGE-NTH-DIGIT-TO ... )
Form:  ( TEST-BODY-CONTRACTS CHANGE-NTH-DIGIT-TO... ) 
Form:  ( TEST-FUNCTION-CONTRACT CHANGE-NTH-DIGIT-TO ...) 
Testing: Done 
Elapsed Run Time: 1.28 seconds
Form:  ( ADMIT-DEFINITION CHANGE-NTH-DIGIT-TO ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT CHANGE-NTH-DIG

### TODO-3 (10 points)

Note that it is impossible for the checksum to be unique. There are only 10 different checksums, and there are far more than 10 possible ticket numbers. So we can't say that if we start with a valid ticket number, but somehow that number is garbled during transmission, we will definitely be able to detect that error. If we're unlucky (and it seems that is a 10% probability), we'll simply end up with another valid ticket number, even after a transmission failure. So the best we can hope for is that if *one* error happens during transmission, we should be able to detect that.

Let's focus on errors of the first type, i.e., when a single digit is randomly changed. Suppose that as start with a valid ticket $A = a_1 a_2 \dots a_{15}$ and end up with $B = a_1 a_2 \dots b_i \dots a_{15}$, where $a_i$ was replaced with $b_i$. We would like to show that ticket $B$ must be invalid. 

Use `test?` to show this property holds (but see next Note). Use `change-nth-digit-to` to create $B$ from an arbitrary ticket $A$, and keep in mind that the only valid index values `k` must be between 0 and 14, since airline tickets have precisely 15 digits.

> Note: Unfortunately, this scheme for validating airline ticket numbers isn't strong enough to catch all 1-digit errors. For example, if the first digit is changed from a 2 to a 9, this does not affect the ticket number modulo 7, so the same checksum will work! The best we can hope for here is that `test?` will show up some counterexamples that will help us understand the limitations of our scheme, and maybe come up with a different scheme if necessary.

> Note: Note also that you can get very "unlucky" in that you end up replacing the kth digit with a digit d that just happens to be the same as the old digit at the kth position. So your condition that you're trying to test should take this into account!

In [3]:
(defsnapshot todo-3)

(test? (implies (and ;; (digit-listp ticket) 
                     (airline-ticketp ticket)
                     ;; (<= (len ticket) 15)
                     (valid-ticket ticket)
                     (natp k)
                     (< k 15)
                     (digitp d)
                     (not (equal (change-nth-digit-to ticket k d) ticket)))
                (not (valid-ticket (change-nth-digit-to ticket k d)))))

ACL2S !>>(DEFSNAPSHOT TODO-3)

Summary
Form:  ( DEFLABEL TODO-3 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-3
ACL2S !>>(TEST?
             (IMPLIES (AND (AIRLINE-TICKETP TICKET)
                           (VALID-TICKET TICKET)
                           (NATP K)
                           (< K 15)
                           (DIGITP D)
                           (NOT (EQUAL (CHANGE-NTH-DIGIT-TO TICKET K D)
                                       TICKET)))
                      (NOT (VALID-TICKET (CHANGE-NTH-DIGIT-TO TICKET K D)))))

**Summary of Cgen/testing**
We tested 45 examples across 1 subgoals, of which 18 (18 unique) satisfied
the hypotheses, and found 3 counterexamples and 15 witnesses.

We falsified the conjecture. Here are counterexamples:
 [found in : "top"]
 -- ((D 7) (K 11) (TICKET '(4 2 0 0 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((D 7) (K 12) (TICKET '(6 4 4 0 7 0 0 0 0 0 0 0 0 0 0)))
 -- ((D 7) (K 5) (TICKET '(2 8 7 7 0 0 0 0 0 0 0 0 0 0 0)))

C

### TODO-4 (10 points)

Now let's try the srecond type of error. Again, if we start with a valid ticket $A = a_1 a_2 \dots a_{15}$ then transpose two digits to end up with ticket $B = a_1 a_2 \dots a_{i+1} a_i \dots a_{15}$, we would like to believe that ticket $B$ is invalid. Use `test?` to verify this property.  Your answer should be very similar to TODO-3.

> Note: Again, this conjecture is not true, so the real exercise is to see if the randomized testing helps us to understand the true properties of our code. In fact, suppose we transpose $a_i$ and ${i+1}$. Using properties of modulos (make sure you understand why), we see that $B$ will have the same checksum as $A$ if $10a_i + a_{i+1} \equiv 10a_{i+1} + a_i \pmod 7$. With a little algebra (make sure you know how), we can show that this is true precisely when $a_i \equiv b_i \pmod 7$, e.g., when we're transposing 07, 18, 29, 70, 81, or 92. In other words, the checksum is valid 6% of the time!

In [4]:
(defsnapshot todo-4)

(test? (implies (and (airline-ticketp ticket)
                     (valid-ticket ticket)
                     (natp k)
                     (< k 14)
                     (not (equal (transpose-nth-digit ticket k) ticket)))
                (not (valid-ticket (transpose-nth-digit ticket k)))))

ACL2S !>>(DEFSNAPSHOT TODO-4)

Summary
Form:  ( DEFLABEL TODO-4 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-4
ACL2S !>>(TEST?
              (IMPLIES (AND (AIRLINE-TICKETP TICKET)
                            (VALID-TICKET TICKET)
                            (NATP K)
                            (< K 14)
                            (NOT (EQUAL (TRANSPOSE-NTH-DIGIT TICKET K)
                                        TICKET)))
                       (NOT (VALID-TICKET (TRANSPOSE-NTH-DIGIT TICKET K)))))

**Summary of Cgen/testing**
We tested 125 examples across 1 subgoals, of which 9 (9 unique) satisfied
the hypotheses, and found 6 counterexamples and 3 witnesses.

We falsified the conjecture. Here are counterexamples:
 [found in : "top"]
 -- ((K 0) (TICKET '(7 0 0 0 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((K 0) (TICKET '(7 0 7 7 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((K 0) (TICKET '(0 7 7 0 0 0 0 0 0 0 0 0 0 0 0)))

Cases in which the conjecture is true include:
 [found in 

## Bank Routing Numbers

The airline ticket checksum relies on converting a list of 14 digits into a single number, then doing modular arithmetic on that number. Bank routing numbers use a different scheme.

Firat, bank routing numbers are nine-digit numbers. There is no "checksum" per se, but the numbers are chosen such that $A = a_1 a_2 \dots a_9$ is valid precisely when
$$7a_1 + 3a_2 + 9a_3 + 7a_4 + 3a_5 + 9a_6 + 7a_7 + 3a_8 + 9a_9 \equiv 0 \pmod {10}$$
The key fact here is that the digits are weighted.

> Aside: With airline ticket numbers, the digit a_{15} was the remainder of something to do with the first 14 digits, but with bank routing numbers, there is a general equation involving all 9 digits. This distinction is superficial. I.e., we could say that $a_9 \euiv -(\cdots) \pmod {10}$ where $(\cdots)$ is the weighted sum of the first eight digits. Also, in airline ticket numbers we computed a large number from the first 14 digits, whereas with bank routing numbers we are adding the digits but with different weights depending on the position. In fact, the weights idea is more general. Converting the digits to numbers is the same as using weights that are the powers of 10! So you see, the idea behind routing numbers is a generalization of the idea behind airline ticket numbers!

### TODO-5 (10 points)

This idea of using weights is very powerful, and many other checksum ideas use it. So let's start with a function that takes care of the weights. You may recognize the relevant operation as the dot product of two vectors:
$$\langle a_1, a_2, \dots, a_n\rangle \cdot \langle w_1, w_2, \dots, w_n\rangle = \sum_{i=1}^{n} a_i \times w_i$$

Define the function `(dotpr as ws)` in ACL2.


In [5]:
(defsnapshot todo-5)

(definec dotpr (as :digit-list ws :digit-list) :nat
    (if (or (endp as) (endp ws))
        0
        (+ (* (first as) (first ws))
           (dotpr (rest as) (rest ws)))))
           
(check-expect (dotpr '(1 2 2 1 5 0 2 7 8) '(7 3 9 7 3 9 7 3 9)) 160)

ACL2S !>>(DEFSNAPSHOT TODO-5)

Summary
Form:  ( DEFLABEL TODO-5 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-5
ACL2S !>>(DEFINEC DOTPR (AS DIGIT-LIST WS DIGIT-LIST)
                  NAT
                  (IF (OR (ENDP AS) (ENDP WS))
                      0
                      (+ (* (FIRST AS) (FIRST WS))
                         (DOTPR (REST AS) (REST WS)))))

Form:  ( TEST-DEFINITION DOTPR ... )
Form:  ( TEST-BODY-CONTRACTS DOTPR... ) 
Form:  ( TEST-FUNCTION-CONTRACT DOTPR ...) 
Testing: Done 
Elapsed Run Time: 1.16 seconds
Form:  ( ADMIT-DEFINITION DOTPR ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT DOTPR ... )
Time:  0.13 seconds (prove: 0.07, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS DOTPR ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.21 seconds
Function Name : DOTPR 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Body

### TODO-6 (10 points)

Now use your definition of `dotpr` to define `valid-routing-number`. For example,

    (valid-routing-number '(1 2 2 1 5 0 2 7 8)) = T


In [6]:
(defsnapshot todo-6)

(defdata routing-number (list digit digit digit digit digit
                              digit digit digit digit))

(definec valid-routing-number (route :digit-list) :bool
    (and (equal (len route) 9)
         (equal (mod (dotpr route '(7 3 9 7 3 9 7 3 9)) 10) 0)))

(check-expect (valid-routing-number '(1 2 2 1 5 0 2 7 8)) t)

ACL2S !>>(DEFSNAPSHOT TODO-6)

Summary
Form:  ( DEFLABEL TODO-6 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-6
ACL2S !>>(DEFDATA ROUTING-NUMBER
                  (LIST DIGIT DIGIT DIGIT
                        DIGIT DIGIT DIGIT DIGIT DIGIT DIGIT))
 Predicate events...
Form:  ( DEFUN ROUTING-NUMBERP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (ROUTING-NUMBERP ACL2::V1) <= body -- not complete. 
Reasons: 
("Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

 (ROUTING-NUMBERP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules."
 "Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

Form:  ( DEFTHM ROUTING-NUMBER=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-ROUTING-NUMBER-BUILTI

### TODO-7 (10 points)

This should be very similar to TODO-3. Use `test?` to show that if you have a valid routing number, then change a digit in the number, the result is **not** a valid routing number.

In this case, you should not see any counterexamples. Here's why. Suppose we change the first digit, so that $a_1\ne b_1$. Then the sum of the other digits is completely unaffected, so the total sum changes only by $7a_1 - 7b_1 \bmod 10$, where the "7" comes from the weight in the first position. In other words, the checksum does \emph{not} change precisely when $7a_1 \equiv 7b_1 \pmod{10}$. When is that? Since 7 and 10 are relatively prime, this only happens when $a_1 \equiv b_1 \pmod{10}$. (For an explanation, you can multiply both sides of the first equation by $7^{-1} \bmod 10$, and that exists because 7 and 10 are relatively prime. In particular, $3\times7 =21 \equiv 1 \pmod{10}$, so $7^{-1} \bmod 10 = 3$.) So if $a_1$ is changed, the checksum has to change, too. That means that a singled-digit error will *always* be detected. So if you see some counterexamples, there is a bug in your code!

In principle, you could prove this in ACL2 by formalizing the argument in the previous paragraph (and generalizing it to all three different weights.) But that is so much work that I won't ask you to do it, even for extra credit. If you do want to pursue such proofs in ACL2, come see me about undergraduate research. Seriously, I have an idea that could turn this projecty into a nice publication for a student.

> Note: The reason this works is that the weights and the modulo are relatively prime. Since the modulo is 10, this means that the weights 2, 4, 5, 6, and 8 are all bad. For example, if the weight is even, then the checksum will be the same if a 3 is changed to an 8.

In [7]:
(defsnapshot todo-7)

(test? (implies (and (routing-numberp route)
                     (valid-routing-number route)
                     (natp k)
                     (< k 9)
                     (digitp d)
                     (not (equal (change-nth-digit-to route k d) route)))
                (not (valid-routing-number (change-nth-digit-to route k d)))))

ACL2S !>>(DEFSNAPSHOT TODO-7)

Summary
Form:  ( DEFLABEL TODO-7 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-7
ACL2S !>>(TEST?
          (IMPLIES
               (AND (ROUTING-NUMBERP ROUTE)
                    (VALID-ROUTING-NUMBER ROUTE)
                    (NATP K)
                    (< K 9)
                    (DIGITP D)
                    (NOT (EQUAL (CHANGE-NTH-DIGIT-TO ROUTE K D)
                                ROUTE)))
               (NOT (VALID-ROUTING-NUMBER (CHANGE-NTH-DIGIT-TO ROUTE K D)))))

**Summary of Cgen/testing**
We tested 12000 examples across 12 subgoals, of which 937 (937 unique)
satisfied the hypotheses, and found 0 counterexamples and 937 witnesses.

Cases in which the conjecture is true include:
 [found in : "top"]
 -- ((D 2) (K 2) (ROUTE '(0 0 0 0 0 0 0 0 0)))
 -- ((D 8) (K 0) (ROUTE '(1 7 7 7 0 0 0 0 0)))
 -- ((D 1) (K 4) (ROUTE '(4 0 1 7 7 7 0 0 0)))

Test? succeeded. No counterexamples were found.

### TODO-8 (10 points)

This should be very similar to TODO-4. Use `test?` to show that if you have a valid routing number, then trnspose two adjacent digits in the number, the result is **not** a valid routing number.

Should you expect any counterexamples? Think of the difference between $7a_1+3a_2$ and $7a_2+3a_1$ mod 10? Well, suppose $(7a_1+3a_2) - (7a_2+3a_1) \equiv 0 \pmod {10}$. With some algebra, you can see that this is the same as
$(7-3)(a_1-a_2) = 4(a_1-a_2) \equiv 0 \pmod {10}$. Since 4 is even, this happens exactly when $a_1-a_2$ is a
multiple of 5, e.g., when $a_1=3$ and $a_2=8$, or about 10% of the time. Again, the question is whether randomized testing lets us see the properties of our code.

> Note: This same argument works whenever the difference between adjacent weights is even. In those cases, there is no way to detect a transposition of digits that differ by 5.



In [8]:
(defsnapshot todo-8)

(test? (implies (and (routing-numberp route)
                     (valid-routing-number route)
                     (natp k)
                     (< k 8)
                     (not (equal (transpose-nth-digit route k) route)))
                (not (valid-routing-number (transpose-nth-digit route k)))))

ACL2S !>>(DEFSNAPSHOT TODO-8)

Summary
Form:  ( DEFLABEL TODO-8 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-8
ACL2S !>>(TEST?
            (IMPLIES
                 (AND (ROUTING-NUMBERP ROUTE)
                      (VALID-ROUTING-NUMBER ROUTE)
                      (NATP K)
                      (< K 8)
                      (NOT (EQUAL (TRANSPOSE-NTH-DIGIT ROUTE K)
                                  ROUTE)))
                 (NOT (VALID-ROUTING-NUMBER (TRANSPOSE-NTH-DIGIT ROUTE K)))))

**Summary of Cgen/testing**
We tested 3149 examples across 4 subgoals, of which 38 (38 unique)
satisfied the hypotheses, and found 3 counterexamples and 35 witnesses.

We falsified the conjecture. Here are counterexamples:
 [found in : "top"]
 -- ((K 0) (ROUTE '(7 2 4 7 0 0 0 0 0)))
 -- ((K 2) (ROUTE '(6 0 5 0 0 7 0 0 0)))
 [found in : "Subgoal 1.9''"]
(IMPLIES (AND (INTEGERP ROUTE17)
              (<= 0 ROUTE17)
              (INTEGERP ROUTE15)
              (<= 0 R

## Credit Cards

Many credit cards use the following checksum scheme. The credit card number is 16 digits, $A=a_1 a_2 \dots a_{16}$ and these are summed using weights, as before. The weights are $2, 1, 2, 1, \dots, 2, 1$. As before, the last digit is chosen so that the weighted sum (almost) is congruent to 0 modulo 10. But there's a twist. It's not just the sum that matters, it's actually
$$2a_1 + a_2 + 2a_3 + a_4 + \dots + 2a_{15} + a_{16} + \text{number of $a_{\text{odd } i}$ that are $\ge 5$}$$
What a weird twist! The number of $a_i$ (where $i$ is odd) that are $\ge 5$. For example, consider the credit card number $4000~0012~3456~7899$. The $a_i$ for odd $i$ are $40~01~35~79$, and the number of those that are $\ge 5$ is 3.

> Aside: If you took COSC 1010 at UW, you may remember this seemingly crazy validation scheme. Implementing this scheme in Java was a favorite exercise of the recently retired but long-serving COSC 1010 instructor at UW! First-year students can write the validation code, though they probably never guessed *why* it was defined this way. As you can probably guess by now, the reason has to do with modular arithmetic!

So why this crazy scheme? As we saw earlier, using weights that are even means that a single digit $a_i$ can be replaced by another digit $b_i$ such that $|a_i-b_i|=5$ and this change goes undetected modulo 10. But, that's where the "$\ge 5$" component comes in. If $|a_i-b_i|=5$, precisely one of them is $\ge 5$, so if one is substituted for the other, the number of digits $\ge 5$ changes by 1. And notice that we only count the digits in odd locations, *which are precisely the digits that have an even weight (i.e., 2).* That's why this scheme works to detect single-digit errors, even though we use even weights.

But what about a transposition error? Suppose that $a_1$ and $a_2$ are swapped. Then the weight will be the same if $(2a_1 + a_2) - (2a_2 + a_1) = a_1 - a_2 \equiv 0 \pmod {10}$, which means that we can see a difference modulo 10 whenever $a_1 \ne a_2$. But what about the $\ge 5$ count? If $a_1 \not\ge 5$ but $a_2 \ge 5$, then the count of digits in odd positions will increase by one. Can that offset the change of the digit sum modulo 10? Well, yes. The count increased by 1, so to offset it the sum needs to be $(2a_1 + a_2) - (2a_2 + a_1) = a_1 - a_2 \equiv -1 \pmod {10}$. Remember that $-1 \bmod 10 = 9$, so this means that an adjacent $90$ can be swapped with $09$ and be undetected.

This crazy checksum scheme is successful in catching all single-digit errors *and* almost all (98%) transpositions!

### TODO-9 (10 points)

Define `valid-ccard-number`, just like in TODO-6. For example,

    (valid-ccard-number '(4 0 0 0 0 0 1 2 3 4 5 6 7 8 9 9)) = t

In [9]:
(defsnapshot todo-9)

(defdata ccard-number (list digit digit digit digit digit
                            digit digit digit digit digit
                            digit digit digit digit digit
                            digit))
                            
(definec count-odds->=-5 (ccard :digit-list) :nat
    (if (endp ccard)
        0
        (if (endp (rest ccard))
            (if (>= (first ccard) 5)
                1
                0)
            (+ (if (>= (first ccard) 5)
                   1
                   0)
               (count-odds->=-5 (rest (rest ccard)))))))


(definec valid-ccard-number (ccard :digit-list) :bool
    (and (equal (len ccard) 16)
         (equal (mod (+ (dotpr ccard '(2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1))
                        (count-odds->=-5 ccard))
                     10) 
                0)))

(check-expect (valid-ccard-number '(4 0 0 0 0 0 1 2 3 4 5 6 7 8 9 9)) t)

ACL2S !>>(DEFSNAPSHOT TODO-9)

Summary
Form:  ( DEFLABEL TODO-9 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-9
ACL2S !>>(DEFDATA CCARD-NUMBER
                  (LIST DIGIT DIGIT DIGIT DIGIT
                        DIGIT DIGIT DIGIT DIGIT DIGIT DIGIT
                        DIGIT DIGIT DIGIT DIGIT DIGIT DIGIT))
 Predicate events...
Form:  ( DEFUN CCARD-NUMBERP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (CCARD-NUMBERP ACL2::V1) <= body -- not complete. 
Reasons: 
("Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

 (CCARD-NUMBERP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules."
 "Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

Form:  ( DEFTHM CCARD-NUMBER=>DEF ...)
 Enume

### TODO-10 (10 points)

This should be very similar to TODO-7. Use `test?` to show that if you have a valid credit card number, then change a digit in the number, the result is **not** a valid credit card number. As we discussed above, you should see no counterexamples. I am not asking you to prove this, just to validate it using random testing with `test?`.

In [10]:
(defsnapshot todo-10)

(test? (implies (and (ccard-numberp ccard)
                     (valid-ccard-number ccard)
                     (natp k)
                     (< k 16)
                     (digitp d)
                     (not (equal (change-nth-digit-to ccard k d) ccard)))
                (not (valid-ccard-number (change-nth-digit-to ccard k d)))))

ACL2S !>>(DEFSNAPSHOT TODO-10)

Summary
Form:  ( DEFLABEL TODO-10 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-10
ACL2S !>>(TEST?
            (IMPLIES
                 (AND (CCARD-NUMBERP CCARD)
                      (VALID-CCARD-NUMBER CCARD)
                      (NATP K)
                      (< K 16)
                      (DIGITP D)
                      (NOT (EQUAL (CHANGE-NTH-DIGIT-TO CCARD K D)
                                  CCARD)))
                 (NOT (VALID-CCARD-NUMBER (CHANGE-NTH-DIGIT-TO CCARD K D)))))


ACL2 Error in ( THM ...):  Out of time in linear arithmetic (add-poly).


**Summary of Cgen/testing**
We tested 3000 examples across 3 subgoals, of which 128 (128 unique)
satisfied the hypotheses, and found 0 counterexamples and 128 witnesses.

Cases in which the conjecture is true include:
 [found in : "top"]
 -- ((D 9) (K 7) (CCARD '(5 7 7 7 0 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((D 2) (K 0) (CCARD '(7 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0)))
 

### TODO-11 (10 points)

This should be very similar to TODO-8. Use `test?` to show that if you have a valid credit card number, then transpose two adjacent digits in the number, the result is not a valid credit card number. As discussed above you should only see counterexamples when the two numbers transposed are 09 or 90.

In [11]:
(defsnapshot todo-11)

(test? (implies (and (ccard-numberp ccard)
                     (valid-ccard-number ccard)
                     (natp k)
                     (< k 15)
                     (not (equal (transpose-nth-digit ccard k) ccard)))
                (not (valid-ccard-number (transpose-nth-digit ccard k)))))

ACL2S !>>(DEFSNAPSHOT TODO-11)

Summary
Form:  ( DEFLABEL TODO-11 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-11
ACL2S !>>(TEST?
          (IMPLIES (AND (CCARD-NUMBERP CCARD)
                        (VALID-CCARD-NUMBER CCARD)
                        (NATP K)
                        (< K 15)
                        (NOT (EQUAL (TRANSPOSE-NTH-DIGIT CCARD K)
                                    CCARD)))
                   (NOT (VALID-CCARD-NUMBER (TRANSPOSE-NTH-DIGIT CCARD K)))))


ACL2 Error in ( THM ...):  Out of time in linear arithmetic (add-poly).


**Summary of Cgen/testing**
We tested 3000 examples across 3 subgoals, of which 23 (23 unique)
satisfied the hypotheses, and found 0 counterexamples and 23 witnesses.

Cases in which the conjecture is true include:
 [found in : "top"]
 -- ((K 0) (CCARD '(4 0 7 7 0 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((K 2) (CCARD '(8 8 7 0 0 0 0 0 0 0 0 0 0 0 0 0)))
 -- ((K 0) (CCARD '(7 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0)))

Test

## ISBNs: Full Error Detection

As we've seen in the past two examples, the "weighted sum mod 10" approach requires that

* weights must not be even or 5 
* adjacent weights differ by an odd amount other than 5.

In both cases, the problem is that weights and adjacent differences must be **relatively prime** to 10, and that is so that $a^{-1} \pmod {10}$ exists.

But you know of another way of ensuring that the weights and adjacent differences are relatively prime? Stop using modulo 10, and instead use modulo a **prime number** $N$. Then all weights and differences are (by definition) relatively prime to $N$. And it just so happens that $11$ is a prime number that's already close to 10!

ISBN numbers (i.e., identifiers for books) follow this approach. An ISBN has 10 digits $A = a_1 a_2 \dots a_{10}$ and the tenth digit is chosen so that the weighted sum with the weights $\langle 10, 9, 8, \dots, 1\rangle$ is equal to $0\pmod{11}$. Since we're doing modulo 11, it is possible that the last "digit" must be 10, so the ISBN format allows the last digit to be 0-9 or X. For example, my favorite book of all time has ISBN 198480278X.

> Aside: There is a newer ISBN-13 format used for books and other media, but the ISBN code we're describing above is the older ISBN-10. It happens that ISBN-13 doesn't have as good error detection properties, because it went back to "mod 10".

### TODO-12 (10 points)

Define the function `(valid-isbn isbn)` that checks to see if an ISBN is actually valid. You will also have to define the data type `isbn`, which consists of a list of 9 digits and a "digit-or-X". For example,

    (valid-isbn '(1 9 8 4 8 0 2 7 8 X)) = t

In [12]:
(defsnapshot todo-12)

(defdata digit-or-x (oneof :digit 'x))

(defdata isbn (list digit digit digit digit digit
                    digit digit digit digit digit-or-x))
                    
(defdata digit-or-x-list (listof digit-or-x))

(definec dotprx (as :digit-or-x-list ws :nat-list) :nat
    (if (or (endp as) (endp ws))
        0
        (+ (* (if (equal (first as) 'x) 10 (first as))
              (first ws))
           (dotprx (rest as) (rest ws)))))

(definec valid-isbn (isbn :digit-or-x-list) :bool
    (and (equal (len isbn) 10)
         (equal (mod (dotprx isbn '(10 9 8 7 6 5 4 3 2 1)) 11) 0)))

(check-expect (valid-isbn '(1 9 8 4 8 0 2 7 8 X)) t)


ACL2S !>>(DEFSNAPSHOT TODO-12)

Summary
Form:  ( DEFLABEL TODO-12 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-12
ACL2S !>>(DEFDATA DIGIT-OR-X (ONEOF DIGIT 'X))
 Predicate events...
Form:  ( DEFUN DIGIT-OR-XP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: DIGIT-OR-XP relatively complete for Tau.
Form:  ( DEFTHM DEF=>DIGIT-OR-X ...)
Form:  ( DEFTHM DIGIT-OR-X=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-DIGIT-OR-X-BUILTIN ...)
Form:  ( DEFUN NTH-DIGIT-OR-X/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.05 seconds (prove: 0.02, print: 0.00, other: 0.04)
 Registering type...
Form:  ( DEFUN NTH-DIGIT-OR-X ...)
Form:  ( ENCAPSULATE (((NTH-DIGIT-OR-X * ...) ...) ...) ...)
Form:  ( DEFUN NTH-DIGIT-OR-X/ACC ...)
Form:  ( ENCAPSULATE (((NTH-DIGIT

### TODO-13 (10 points)


This should be very similar to TODO-7. Use test? to show that if you have a valid isbn, then change a digit in the isbn, the result is not a valid isbn. As we discussed above, you should see no counterexamples, since we are using "mod 11" so all weights are relatively prime to the modulus. I am not asking you to prove this, just to validate it using random testing with `test?`.


In [13]:
(defsnapshot todo-13)

(definec change-nth-digit-or-x-to (msg :digit-or-x-list n :nat d :digit-or-x) :digit-or-x-list
    (if (endp msg)
        msg
        (if (zp n)
            (cons d (rest msg))
            (cons (first msg) (change-nth-digit-or-x-to (rest msg) (1- n) d)))))

(test? (implies (and (isbnp isbn)
                     (valid-isbn isbn)
                     (natp k)
                     (< k 10)
                     (digit-or-xp d)
                     (not (equal (change-nth-digit-or-x-to isbn k d) isbn)))
                (not (valid-isbn (change-nth-digit-or-x-to isbn k d)))))

ACL2S !>>(DEFSNAPSHOT TODO-13)

Summary
Form:  ( DEFLABEL TODO-13 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-13
ACL2S !>>(DEFINEC CHANGE-NTH-DIGIT-OR-X-TO
                  (MSG DIGIT-OR-X-LIST N NAT D DIGIT-OR-X)
                  DIGIT-OR-X-LIST
                  (IF (ENDP MSG)
                      MSG
                      (IF (ZP N)
                          (CONS D (REST MSG))
                          (CONS (FIRST MSG)
                                (CHANGE-NTH-DIGIT-OR-X-TO (REST MSG)
                                                          (1- N)
                                                          D)))))

Form:  ( TEST-DEFINITION CHANGE-NTH-DIGIT-OR-X-TO ... )
Form:  ( TEST-BODY-CONTRACTS CHANGE-NTH-DIGIT-OR-X-TO... ) 
Form:  ( TEST-FUNCTION-CONTRACT CHANGE-NTH-DIGIT-OR-X-TO ...) 
Testing: Done 
Elapsed Run Time: 1.30 seconds
Form:  ( ADMIT-DEFINITION CHANGE-NTH-DIGIT-OR-X-TO ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, ot

### TODO-14 (10 points)

This should be very similar to TODO-8. Use `test?` to show that if you have a valid isbn, then transpose two adjacent digits in the isbn, the result is not a valid isbn. As discussed above you should not see any counterexamples since we're using "mod 11" which is prime. I am not asking you to prove this, just to validate it using random testing with `test?`.

In [14]:
(defsnapshot todo-14)

(definec transpose-nth-digit-or-x (msg :digit-or-x-list n :nat) :digit-or-x-list
    (if (or (endp msg) (endp (rest msg)))
        msg
        (if (zp n)
            (cons (second msg) 
                  (cons (first msg) 
                        (rest (rest msg))))
            (cons (first msg) (transpose-nth-digit-or-x (rest msg) (1- n))))))

(test? (implies (and (isbnp isbn)
                     (valid-isbn isbn)
                     (natp k)
                     (< k 9)
                     (not (equal (transpose-nth-digit-or-x isbn k) isbn)))
                (not (valid-isbn (transpose-nth-digit-or-x isbn k)))))

ACL2S !>>(DEFSNAPSHOT TODO-14)

Summary
Form:  ( DEFLABEL TODO-14 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-14
ACL2S !>>(DEFINEC TRANSPOSE-NTH-DIGIT-OR-X
                  (MSG DIGIT-OR-X-LIST N NAT)
                  DIGIT-OR-X-LIST
                  (IF (OR (ENDP MSG) (ENDP (REST MSG)))
                      MSG
                      (IF (ZP N)
                          (CONS (SECOND MSG)
                                (CONS (FIRST MSG) (REST (REST MSG))))
                          (CONS (FIRST MSG)
                                (TRANSPOSE-NTH-DIGIT-OR-X (REST MSG)
                                                          (1- N))))))

Form:  ( TEST-DEFINITION TRANSPOSE-NTH-DIGIT-OR-X ... )
Form:  ( TEST-BODY-CONTRACTS TRANSPOSE-NTH-DIGIT-OR-X... ) 
Form:  ( TEST-FUNCTION-CONTRACT TRANSPOSE-NTH-DIGIT-OR-X ...) 
Testing: Done 
Elapsed Run Time: 1.56 seconds
Form:  ( ADMIT-DEFINITION TRANSPOSE-NTH-DIGIT-OR-X ... )
Time:  0.03 seconds (prove: 0.

## Postal Barcodes: Error Detection *and* Correction

We've seen how we can detect all one-digit and transposition errors using a weighted sum modulo a prime number, and the cost is simply having an extra digit in the message. If we allow even more overhead (i.e., a larger checksum), we can detect **and correct** these errors.

The code we're going to explore is from the post office. You may have gotten mail with a yellow sticker attached, and in that sticker is a barcode with long and short lines&mdash;a list of ones and zeros, if you will. These are used to encode important information in a machine-readable format. In particular, the barcode has four parts:

1. The Zip code
2. The extended 4-digit portion of the zip code
3. a 2-digit code for presorting mail, e.g., a portion of the house number
4. a checksum to make the sunm of all 12 digits equal to 0 modulo 10

For example, a code to my home address may look like `82063 9201 18 0`.

Wait a minute. This system is using "mod 10" and doesn't have any weights, so how can it be able to detect and correct errors?

The answer is that the digits themselves are encoded into binary. So instead of having a list of 12 digits, it is actually a list of 60 bits. The encoding of digits to bits is as follows:


| Digit   | Bits   | Digit   | Bits   | Digit   | Bits   | Digit   | Bits   | Digit   | Bits   |
| ------- | ------ | ------- | ------ | ------- | ------ | ------- | ------ | ------- | ------ |
| 0       | 11000  | 1       | 00011  | 2       | 00101  | 3       | 00110  | 4       | 01001  |
| 5       | 01010  | 6       | 01100  | 7       | 10001  | 8       | 10010  | 9       | 10100  |

Notice we are using five bits to encode each digit, even though only four bits are necessary. That is the extra overhead we're paying to get error detection and correction.

How does this work? First, notice that all 5-bit code have three 0s and two 1s. If you are writing this code in December, then you can answer the question, "How many ways can you write down exactly three 0s and two 1s?". (Hint: It's 5!/3!2! = 10.) So there is a unique code for each digit.

Now, suppose a single bit flips from 0 to 1 or 1 to 0. Then we can tell exactly which digit is encoded incorrectly, since that digit will no longer have exactly three 0s and two 1s. Suppose we know that the first digit has an error. Then we can use the "mod 10" checksum to find what that digit must be! (Realize that the weights are all 1s, which are relatively prime to 10, so single errors in the **digits** can be detected.) Since we know what the right digit should have been, it's easy to replace the five bits that correspond to that digit with the correct five bits.

What about transposition errors? Suppose a transposition occurs within the five bits for a given digit. Then the digit will change to another *valid* digit. (Remember, there are exactly ten bit patterns with threee 0s and two 1s, so swapping two bits will result in another one of those ten bit patterns.) The "mod 10" checksum will detect that there's an error, but there is no way to discover which of the digits is wrong. So this error is not corrected. On the other hand, if a transposition occurs at the boundary of two digits, then both digits will have invalid 5-bit patterns. That means we know which two bits were transposed (at the boundary of the two illegal digits), and we can swap them back to correct the error.

So this scheme detects all one-bit and transposition errors, and corrects all one-bit errors and many transposition errors. Since this is intended for computer communication, one-bit errors are common but transposition errors are not. (Transposition errors are common to humans&mdash;just see all the typos in this assignment, for example. But computers just don't make those kind of mistakes. On the other hand, computers are more likely to have a one-bit error during transmission, due to faulty wiring, cosmic rays, disk failures, etc.) So this is a very effective error detection and (mostly) correction scheme! 


### TODO-15 (10 points)

Define the functions `(encode digits)` and `(decode bits)` that convert a list of digits to a list of 5-bit words and vice versa. For example,

    (encode '(8 2 0 6 3)) = '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))
    (decode '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = '(8 2 0 6 3)

In preparation for the rest of this assignment, have `decode` return the digit 0 for any 5-bit word that it does not recognize.

Of course, you will have to define any relevant types with `defdata`.

> Note: I chose to store the bits as a list of words to make it easier to write the encode and decode functions. It really should be a list of bits!

> Hint: There are more clever ways of doing this, but I just used a (deeply) nested if-statement to convert from individual digits to words and vice versa.


In [15]:
(defsnapshot todo-15)

(defdata bit (oneof 0 1))
(defdata word (list bit bit bit bit bit))
(defdata word-list (listof word))

(definec encode-digit (digit :digit) :word
    (if (= digit 1)
        '(0 0 0 1 1)
        (if (= digit 2)
            '(0 0 1 0 1)
            (if (= digit 3)
                '(0 0 1 1 0)
                (if (= digit 4)
                    '(0 1 0 0 1)
                    (if (= digit 5)
                        '(0 1 0 1 0)
                        (if (= digit 6)
                            '(0 1 1 0 0)
                            (if (= digit 7)
                                '(1 0 0 0 1)
                                (if (= digit 8)
                                    '(1 0 0 1 0)
                                    (if (= digit 9)
                                        '(1 0 1 0 0)
                                        '(1 1 0 0 0)
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

(definec encode (digits :digit-list) :word-list
    (if (endp digits)
        nil
        (cons (encode-digit (first digits))
              (encode (rest digits)))))

(check-expect (encode '(8 2 0 6 3)) '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)))

(definec decode-word (word :word) :digit
    (if (equal word '(0 0 0 1 1))
        1
        (if (equal word '(0 0 1 0 1))
            2
            (if (equal word '(0 0 1 1 0))
                3
                (if (equal word '(0 1 0 0 1))
                    4
                    (if (equal word '(0 1 0 1 0))
                        5
                        (if (equal word '(0 1 1 0 0))
                            6
                            (if (equal word '(1 0 0 0 1))
                                7
                                (if (equal word '(1 0 0 1 0))
                                    8
                                    (if (equal word '(1 0 1 0 0))
                                        9
                                        0
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

(definec decode (words :word-list) :digit-list
    (if (endp words)
        nil
        (cons (decode-word (first words))
              (decode (rest words)))))

(check-expect (decode '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) '(8 2 0 6 3))




ACL2S !>>(DEFSNAPSHOT TODO-15)

Summary
Form:  ( DEFLABEL TODO-15 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-15
ACL2S !>>(DEFDATA BIT (ONEOF 0 1))
 Predicate events...
Form:  ( DEFTHM BITP-TESTTHM ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: BITP relatively complete for Tau.
Form:  ( DEFTHM ACL2::DEF=>BIT ...)
Form:  ( DEFTHM ACL2::BIT=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-BIT-BUILTIN ...)
Form:  ( DEFUN NTH-BIT/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.07 seconds (prove: 0.03, print: 0.00, other: 0.04)
 Registering type...
Form:  ( DEFUN NTH-BIT ...)
Form:  ( ENCAPSULATE (((NTH-BIT * ...) ...) ...) ...)
Form:  ( DEFUN NTH-BIT/ACC ...)
Form:  ( ENCAPSULATE (((NTH-BIT/ACC * ...) ...) ...) ...)
Form:  ( DEFATTACH (NTH-BIT N

### TODO-16 (10 points)

It should be obviout, right?, that if you encode a message and then decode the result, you get the original message back. Use `thm` to prove this idea in ACL2.



In [16]:
(defsnapshot todo-16)

(thm (implies (digit-listp message)
              (equal (decode (encode message))
                     message)))

ACL2S !>>(DEFSNAPSHOT TODO-16)

Summary
Form:  ( DEFLABEL TODO-16 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-16
ACL2S !>>(THM (IMPLIES (DIGIT-LISTP MESSAGE)
                       (EQUAL (DECODE (ENCODE MESSAGE))
                              MESSAGE)))

ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


Name the formula above *1.

Perhaps we can prove *1 by induction.  Three induction schemes are
suggested by this conjecture.  Subsumption reduces that number to two.
These merge into one derived induction scheme.  

We will induct according to a scheme suggested b

### TODO-17 (10 points)

Now it is time to detect one-bit errors. Write a function called `(detect encoded-msg)` that takes in a list of 5-bit words and returns the (zero-based) index of the first digit that has an invalid encoding. Remember that an invalid encoding is one of the ones that corresponds to a digit. If all the digits are valid, the function should return -1.  For example,

    (detect '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = -1
    (detect '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = 2



In [17]:
(defsnapshot todo-17)

(definec valid-word (word :word) :bool
    (if (member-equal word '((0 0 0 1 1)
                       (0 0 1 0 1)
                       (0 0 1 1 0)
                       (0 1 0 0 1)
                       (0 1 0 1 0)
                       (0 1 1 0 0)
                       (1 0 0 0 1)
                       (1 0 0 1 0)
                       (1 0 1 0 0)
                       (1 1 0 0 0)))
        t
        nil))

(definec detect (encoded-msg :word-list) :int
    (if (endp encoded-msg)
        -1
        (if (valid-word (first encoded-msg))
            (if (< (detect (rest encoded-msg)) 0)
                -1
                (1+ (detect (rest encoded-msg))))
            0)))

(check-expect (detect '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) 2)

ACL2S !>>(DEFSNAPSHOT TODO-17)

Summary
Form:  ( DEFLABEL TODO-17 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-17
ACL2S !>>(DEFINEC VALID-WORD (WORD WORD)
                  BOOL
                  (IF (MEMBER-EQUAL WORD
                                    '((0 0 0 1 1)
                                      (0 0 1 0 1)
                                      (0 0 1 1 0)
                                      (0 1 0 0 1)
                                      (0 1 0 1 0)
                                      (0 1 1 0 0)
                                      (1 0 0 0 1)
                                      (1 0 0 1 0)
                                      (1 0 1 0 0)
                                      (1 1 0 0 0)))
                      T NIL))

Form:  ( TEST-DEFINITION VALID-WORD ... )
Form:  ( TEST-BODY-CONTRACTS VALID-WORD... ) 
Form:  ( TEST-FUNCTION-CONTRACT VALID-WORD ...) 
Testing: Done 
Elapsed Run Time: 0.12 seconds
Form:  ( ADMIT-DEFINITION VA

### TODO-18 (10 points)

We will need a version of `change-nth-digit-to` that works on bits. Define the function `(change-nth-bit encoded-msg k n)` that flips the nth bit of the kth word in `encocded-msg`. E.g.,'

    (change-nth-bit '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 2 1)
                  = '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0))

Note that since the bits can only be 0 or 1, there is no need to specify what the new value of the bit should be! Also, the result of calling `change-nth-bit` with legal arguments (i.e., `k` and `n` in range) will always be different than the original message. This will simplify the way we need to state some properties later.

In [18]:
(defsnapshot todo-18)

(definec flip-if (bit :bit n :nat target :nat) :bit
    (if (equal n target)
        (- 1 bit)
        bit))

(definec change-nth-bit-in-word (word :word n :nat) :word
    (list (flip-if (first word) n 0)
          (flip-if (second word) n 1)
          (flip-if (third word) n 2)
          (flip-if (fourth word) n 3)
          (flip-if (fifth word) n 4)))

(definec change-nth-bit (words :word-list k :nat n :nat) :word-list
    (if (endp words)
        nil
        (if (zp k)
            (cons (change-nth-bit-in-word (first words) n)
                  (rest words))
            (cons (first words)
                  (change-nth-bit (rest words) (1- k) n)))))

(check-expect (change-nth-bit '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 2 1)
              '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0)))

ACL2S !>>(DEFSNAPSHOT TODO-18)

Summary
Form:  ( DEFLABEL TODO-18 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-18
ACL2S !>>(DEFINEC FLIP-IF (BIT BIT N NAT TARGET NAT)
                  BIT (IF (EQUAL N TARGET) (- 1 BIT) BIT))

Form:  ( TEST-DEFINITION FLIP-IF ... )
Form:  ( TEST-BODY-CONTRACTS FLIP-IF... ) 
Form:  ( TEST-FUNCTION-CONTRACT FLIP-IF ...) 
Testing: Done 
Elapsed Run Time: 0.45 seconds
Form:  ( ADMIT-DEFINITION FLIP-IF ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT FLIP-IF ... )
Time:  0.12 seconds (prove: 0.05, print: 0.00, other: 0.07)
Form:  ( PROVE-BODY-CONTRACTS FLIP-IF ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.19 seconds
Function Name : FLIP-IF 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Body Contracts proven ----- [*]
 T
ACL2S !>>(DEFINEC CHANGE-NTH-BIT-IN-WORD (WORD WORD N NAT)
                  WORD
            

### TODO-19 (10 points)

Suppose we have a message $M$, then convert to a list of 5-bit words $A$. We take a copy of that (encoded) message $A$ to another user, and possibly the copy $A'$ has a one-bit error. (You should be thinkinng about some nested calls of functions you already defined in ACL2.)

Define a function called `(correct encoded-msg-copy)` that decodes the encoded msg into a list of digits, but correcting the `encoded-msg` so that it recovers from a one-bit error. Here's how it does it:

1. First call `detect` to find if one of the digits in `encoded-msg-copy`.
2. Also, convert the `encoded-msg-copy` to a list of digits using `decode`. Note that if there is an error, that 
specific digit will be a 0 (because of the way we wrote `decode`.)
3. If no errors are detected, then the answer is the list of digits that `decode` returned.
4. If an error was made, then compute the correct checksum for the current list. That should be the the number that you need to add to sum of the digits to get 0 (mod 10). E.g., if the digits add up to 17, then the correct checksum is 3.
5. The correct answer is the decode list, but changing the incorrect digit to be this checksum.

In [19]:
(defsnapshot todo-19)


(definec sumlist (list :nat-list) :nat
    (if (endp list)
        0
        (+ (first list) (sumlist (rest list)))))

(definec correct (encoded-msg-copy :word-list) :digit-list
    (if (< (detect encoded-msg-copy) 0)
        (decode encoded-msg-copy)
        (change-nth-digit-to (decode encoded-msg-copy)
                             (detect encoded-msg-copy)
                             (mod (- (sumlist (decode encoded-msg-copy))) 10))))

(check-expect (correct '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0) (0 0 0 1 1))) '(8 2 0 6 3 1))





ACL2S !>>(DEFSNAPSHOT TODO-19)

Summary
Form:  ( DEFLABEL TODO-19 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-19
ACL2S !>>(DEFINEC SUMLIST (LIST NAT-LIST)
                  NAT
                  (IF (ENDP LIST)
                      0
                      (+ (FIRST LIST) (SUMLIST (REST LIST)))))

Form:  ( TEST-DEFINITION SUMLIST ... )
Form:  ( TEST-BODY-CONTRACTS SUMLIST... ) 
Form:  ( TEST-FUNCTION-CONTRACT SUMLIST ...) 
Testing: Done 
Elapsed Run Time: 0.58 seconds
Form:  ( ADMIT-DEFINITION SUMLIST ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT SUMLIST ... )
Time:  0.15 seconds (prove: 0.07, print: 0.00, other: 0.08)
Form:  ( PROVE-BODY-CONTRACTS SUMLIST ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.23 seconds
Function Name : SUMLIST 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Body Contracts proven ----- [*]
 T
ACL2S !>>(DEFINEC
   

### TODO-20 (10 points)

Now, use `test?` to verify that when you encode message $A$, and $A'$ is either equal to $A$ or equal to the result of changing **one** bit in $A$, then when `(correct A')` is equal to $A$. Compare this with TODO-16 which considered the simpler case when $A'$ was always equal to $A$. We assume here that $A$ is a valid message, i.e., the sum of its digits modulo 10 is equal to 0.

So there you have it. This postnet scheme really can recover the original message, even when a single bit is corrupted in transmission (or an envelope is smudged with dirt in one place)!

In [20]:
(defsnapshot todo-20)

(test? (implies (and (digit-listp message)
                     (equal (mod (sumlist message) 10) 0)
                     (natp k)
                     (< k (len message))
                     (natp n)
                     (< n 5)
                     (or (equal encoded-message-copy (encode message))
                         (equal encoded-message-copy (change-nth-bit (encode message) k n))))
                 (equal (correct encoded-message-copy) message)))
                

ACL2S !>>(DEFSNAPSHOT TODO-20)

Summary
Form:  ( DEFLABEL TODO-20 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-20
ACL2S !>>(TEST?
            (IMPLIES (AND (DIGIT-LISTP MESSAGE)
                          (EQUAL (MOD (SUMLIST MESSAGE) 10) 0)
                          (NATP K)
                          (< K (LEN MESSAGE))
                          (NATP N)
                          (< N 5)
                          (OR (EQUAL ENCODED-MESSAGE-COPY (ENCODE MESSAGE))
                              (EQUAL ENCODED-MESSAGE-COPY
                                     (CHANGE-NTH-BIT (ENCODE MESSAGE) K N))))
                     (EQUAL (CORRECT ENCODED-MESSAGE-COPY)
                            MESSAGE)))

**Summary of Cgen/testing**
We tested 8000 examples across 8 subgoals, of which 8 (8 unique) satisfied
the hypotheses, and found 0 counterexamples and 8 witnesses.

Cases in which the conjecture is true include:
 [found in : "Subgoal 2.3"]
 -- ((ENCODED-MESSAGE

## Extra-Credit UPCs (up to 50 points)

Look up how UPC codes are encoded, e.g., in the [Wikipedia entry](https://en.wikipedia.org/wiki/Universal_Product_Code#Check_digit_calculation). Then repeat ther steps todo-6, todo-7, and todo-8, but for UPC codes.

Do you expect counbterexamples in todo-7 and todo-8? Think about our discussions regarding relative prime weights and difference of adjacent weights!

In [24]:
(defsnapshot ec-1)

(definec valid-upc-a (upc-a :digit-list) :bool
    (if (equal (len upc-a) 12)
        (equal (mod (dotpr upc-a '(3 1 3 1 3 1 3 1 3 1 3 1)) 10) 0)
        nil))

(check-expect (valid-upc-a '(1 2 2 1 5 0 2 7 8 0 0 6)) t)
(check-expect (valid-upc-a '(1 1 0 0 0 0 0 0 0 0 9 9)) t)
(check-expect (valid-upc-a '(1 2 2 1 5 8 2 7 0 9 1 0)) t)
(check-expect (valid-upc-a '(1 2 3 1 5 0 2 7 8)) nil)
(check-expect (valid-upc-a '(0 3 6 0 0 0 2 9 1 4 5 2)) t)

(defdata ticketNum
    (range integer (0 <= _ <= 14)))

(definec get-nthDigit (ticket :digit-list n :ticketNum) :digit-list
    (if (< (- (len ticket) 1) n) 
        nil
        (if (equal n 0)
            (list (first ticket))
            (get-nthDigit (rest ticket) (- n 1)))))

(test? (implies (and (digit-listp upc-a) 
                     (natp nthDigit) 
                     (digitp newDigit) 
                     (< nthDigit (len upc-a)) 
                     (equal (len upc-a) 12)
                     (not (equal (get-nthDigit upc-a nthDigit) newDigit))
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (change-nth-digit-to upc-a nthDigit newDigit)) nil)))

;Since adjacent weights are different, most transposition errors should be detectable
(test? (implies (and (digit-listp upc-a) 
                     (natp nthDigit) 
                     (< nthDigit (len upc-a)) 
                     (equal (len upc-a) 12)
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (transpose-nth-digit upc-a nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT EC-1)
          76:x(DEFSNAPSHOT TODO-20)

Summary
Form:  ( DEFLABEL EC-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EC-1
ACL2S !>>(DEFINEC VALID-UPC-A (UPC-A DIGIT-LIST)
                  BOOL
                  (IF (EQUAL (LEN UPC-A) 12)
                      (EQUAL (MOD (DOTPR UPC-A '(3 1 3 1 3 1 3 1 3 1 3 1))
                                  10)
                             0)
                      NIL))

Form:  ( TEST-DEFINITION VALID-UPC-A ... )
Form:  ( TEST-BODY-CONTRACTS VALID-UPC-A... ) 
Form:  ( TEST-FUNCTION-CONTRACT VALID-UPC-A ...) 
Testing: Done 
Elapsed Run Time: 0.18 seconds
Form:  ( ADMIT-DEFINITION VALID-UPC-A ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT VALID-UPC-A ... )
Time:  0.09 seconds (prove: 0.03, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS VALID-UPC-A ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Elapsed Run Time: 0.17